#### Analysis

In [1]:
from typing import List
import os
import pickle
import squarify

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

SAVEPATH = "./figure"
TRACEPATH = "./data/job_trace"
PKLPATH = "./data/utilization/util_pkl"

sns.set_style("ticks")
font = {
    "font.family": "Roboto",
    "font.size": 12,
}
sns.set_style(font)
paper_rc = {
    "lines.linewidth": 3,
    "lines.markersize": 10,
}
sns.set_context("paper", font_scale=2, rc=paper_rc)
cmp = sns.color_palette("tab10")


def autolabel(rects, ax, prec=1):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate(
            f"{height:.{prec}f}",
            xy=(rect.get_x() + rect.get_width() / 2, height),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha="center",
            va="bottom",
            size=16,
        )


def calculate_num_cdf_customized_xaxis(df: pd.DataFrame, x_axis: List, key: str):
    """
    Calculate quantity percentile CDF with customized threshold of x-axis, y-axis: 0-100%,
    """
    # print("Parsing")
    data = df[[key]].copy()
    data.dropna(inplace=True)

    y = [len(data[data[key] <= x]) / len(data) * 100 for x in x_axis]

    return y


def calculate_sum_cdf_customized_xaxis(df: pd.DataFrame, x_axis: List, key: str, key_to_time=None):
    """
    Calculate sum CDF with customized threshold of x-axis, y-axis: 0-100%,
    """
    if key_to_time is not None:
        data = df[[key, key_to_time]].copy()
        data["new"] = data[key] * data[key_to_time]
    else:
        data = df[[key]].copy()
        data["new"] = data[key]
    data.dropna(inplace=True)
    sum = data["new"].sum()

    y = [data[data[key] <= x]["new"].sum() / sum * 100 for x in x_axis]

    return y


if not os.path.exists(SAVEPATH):
    os.makedirs(SAVEPATH)


data_seren = pd.read_csv(f"{TRACEPATH}/trace_seren.csv")
data_kalos = pd.read_csv(f"{TRACEPATH}/trace_kalos.csv")
data_philly = pd.read_csv(f"{TRACEPATH}/trace_previous_work/philly_trace.csv")
data_helios = pd.read_csv(f"{TRACEPATH}/trace_previous_work/helios_trace.csv")
data_pai = pd.read_csv(f"{TRACEPATH}/trace_previous_work/pai_trace.csv")

# A few further process
data_pai.rename(columns={"plan_cpu": "cpu_num", "plan_gpu": "gpu_num", "wait_time": "queue", "status": "state"}, inplace=True)
data_pai[["cpu_num", "gpu_num"]] /= 100
data_pai["state"] = data_pai["state"].map({"Failed": "FAILED"})  # Not suitable for final state analysis
data_philly["state"] = data_philly["state"].map({"Pass": "COMPLETED", "Failed": "FAILED", "Killed": "CANCELLED"})

#### CDF: GPU Job Duration & Utilization

In [ ]:
x = [2**i for i in range(0, 22)]
y_gpu_seren = calculate_num_cdf_customized_xaxis(data_seren[data_seren["gpu_num"] > 0], x_axis=x, key="duration")
y_gpu_kalos = calculate_num_cdf_customized_xaxis(data_kalos[data_kalos["gpu_num"] > 0], x_axis=x, key="duration")
y_gpu_philly = calculate_num_cdf_customized_xaxis(data_philly[data_philly["gpu_num"] > 0], x_axis=x, key="duration")
y_gpu_helios = calculate_num_cdf_customized_xaxis(data_helios[data_helios["gpu_num"] > 0], x_axis=x, key="duration")
y_gpu_pai = calculate_num_cdf_customized_xaxis(data_pai[data_pai["gpu_num"] > 0], x_axis=x, key="duration")

with open(f"{PKLPATH}/util_gpu_seren.pkl", "rb") as file:
    x1, y1, _, _, _, _, _, _, _, _ = pickle.load(file)

with open(f"{PKLPATH}/util_gpu_kalos.pkl", "rb") as file:
    x4, y4, _, _ = pickle.load(file)

with open(f"{PKLPATH}/util_gpu_pai.pkl", "rb") as file:  # Collect via Antman
    x2, y2 = pickle.load(file)

with open(f"{PKLPATH}/util_gpu_philly.pkl", "rb") as file:
    x3, y3 = pickle.load(file)

In [ ]:
linestyles = ["-", "--", ":", ":", ":"]
grid_params = dict(width_ratios=[1, 1])
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, constrained_layout=True, figsize=(9, 3.75))

ax1.plot(x, y_gpu_seren, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax1.plot(x, y_gpu_kalos, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")
ax1.plot(x, y_gpu_philly, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="Philly")
ax1.plot(x, y_gpu_helios, linestyles[3], linewidth=3, alpha=0.9, color=cmp[3], label="Helios")
ax1.plot(x, y_gpu_pai, linestyles[3], linewidth=3, alpha=0.9, color=cmp[4], label="PAI")

ax2.plot(x1, y1, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax2.plot(x4, y4, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")
ax2.plot(x2, y2, linestyles[2], linewidth=3, alpha=0.9, color=cmp[4], label="PAI")
ax2.plot(x3, y3, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="Philly")

ax1.set_xlabel(f"(a) GPU Job Duration (s)")
ax1.set_ylabel(f"CDF (%)")
ax1.set_xscale("log")
ax1.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6])
ax1.set_xlim(1, x[-1])
ax1.set_ylim(-0.5, 100.8)
ax1.grid(linestyle=":")

ax2.set_xlabel(f"(b) GPU Utilization (%)")
ax2.set_ylabel(f"CDF (%)")
ax2.set_xlim(-0.8, 100.8)
ax2.set_xticks([0, 25, 50, 75, 100])
ax2.set_ylim(0, 100.8)
ax2.grid(linestyle=":")

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, ncols=5, bbox_to_anchor=(0.1, 1.145), loc=2, columnspacing=1.5, handletextpad=0.5)

sns.despine()
fig.savefig(f"{SAVEPATH}/cdf_job_duration_util.pdf", bbox_inches="tight")

#### CDF: GPU Number

In [ ]:
x = [i for i in range(0, 1025)]
y_gpu_seren = calculate_num_cdf_customized_xaxis(data_seren[data_seren["gpu_num"] > 0], x_axis=x, key="gpu_num")
y_gpu_kalos = calculate_num_cdf_customized_xaxis(data_kalos[data_kalos["gpu_num"] > 0], x_axis=x, key="gpu_num")
y_gpu_philly = calculate_num_cdf_customized_xaxis(data_philly[data_philly["gpu_num"] > 0], x_axis=x, key="gpu_num")
y_gpu_helios = calculate_num_cdf_customized_xaxis(data_helios[data_helios["gpu_num"] > 0], x_axis=x, key="gpu_num")
y_gpu_pai = calculate_num_cdf_customized_xaxis(data_pai[data_pai["gpu_num"] > 0], x_axis=x, key="gpu_num")

y_gtime_seren = calculate_sum_cdf_customized_xaxis(
    data_seren[data_seren["gpu_num"] > 0], x_axis=x, key="gpu_num", key_to_time="duration"
)
y_gtime_kalos = calculate_sum_cdf_customized_xaxis(
    data_kalos[data_kalos["gpu_num"] > 0], x_axis=x, key="gpu_num", key_to_time="duration"
)
y_gtime_philly = calculate_sum_cdf_customized_xaxis(
    data_philly[data_philly["gpu_num"] > 0], x_axis=x, key="gpu_num", key_to_time="duration"
)
y_gtime_helios = calculate_sum_cdf_customized_xaxis(
    data_helios[data_helios["gpu_num"] > 0], x_axis=x, key="gpu_num", key_to_time="duration"
)
y_gtime_pai = calculate_sum_cdf_customized_xaxis(
    data_pai[data_pai["gpu_num"] > 0], x_axis=x, key="gpu_num", key_to_time="duration"
)

In [ ]:
linestyles = ["-", "--", ":", ":", ":"]
grid_params = dict(width_ratios=[1, 1])
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, constrained_layout=True, figsize=(9, 3.75))

ax1.plot(x, y_gpu_seren, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax1.plot(x, y_gpu_kalos, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")
ax1.plot(x, y_gpu_philly, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="Philly")
ax1.plot(x, y_gpu_helios, linestyles[3], linewidth=3, alpha=0.9, color=cmp[3], label="Helios")
ax1.plot(x, y_gpu_pai, linestyles[3], linewidth=3, alpha=0.9, color=cmp[4], label="PAI")

ax2.plot(x, y_gtime_seren, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax2.plot(x, y_gtime_kalos, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")
ax2.plot(x, y_gtime_philly, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="Philly")
ax2.plot(x, y_gtime_helios, linestyles[3], linewidth=3, alpha=0.9, color=cmp[3], label="Helios")
ax2.plot(x, y_gtime_pai, linestyles[3], linewidth=3, alpha=0.9, color=cmp[4], label="PAI")


ax1.set_xlabel(f"(a) Number of GPU")
ax1.set_ylabel(f"CDF of Jobs (%)")
ax1.set_xscale("log", base=2)
ax1.set_xticks([2**i for i in range(0, 11, 2)])
ax1.set_xticklabels(
    [2**i for i in range(0, 10, 2)]
    + [
        "1024+",
    ]
)
ax1.set_xlim(1, x[-1] + 1)
ax1.set_ylim(-0.5, 100.8)
ax1.grid(linestyle=":")

ax2.set_xlabel(f"(b) Number of GPU")
ax2.set_ylabel(f"CDF of GPU Time (%)")
ax2.set_xscale("log", base=2)
ax2.set_xticks([2**i for i in range(0, 11, 2)])
ax2.set_xticklabels(
    [2**i for i in range(0, 10, 2)]
    + [
        "1024+",
    ]
)
ax2.set_xlim(1, x[-1] + 50)
ax2.set_ylim(-0.5, 100.8)
ax2.grid(linestyle=":")

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, ncols=5, bbox_to_anchor=(0.1, 1.145), loc=2, columnspacing=1.5, handletextpad=0.5)

sns.despine()
fig.savefig(f"{SAVEPATH}/cdf_job_gpunum.pdf", bbox_inches="tight")

#### Bar: Job Final State

In [ ]:
df = pd.read_csv("./data/cluster_summary.csv", index_col="id")
grid_params = dict(width_ratios=[1, 1])
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, constrained_layout=True, figsize=(9, 3.75))

x = np.arange(1, 3)
width = 0.22
p1 = ax1.bar(
    x - width,
    df.loc[["Seren", "Kalos"], "complete_rate_gpu"] * 100,
    width,
    label="Completed",
    alpha=0.8,
    linewidth=1,
    edgecolor="k",
)
p2 = ax1.bar(
    x, df.loc[["Seren", "Kalos"], "cancel_rate_gpu"] * 100, width, label="Canceled", alpha=0.8, linewidth=1, edgecolor="k"
)
p3 = ax1.bar(
    x + width, df.loc[["Seren", "Kalos"], "fail_rate_gpu"] * 100, width, label="Failed", alpha=0.8, linewidth=1, edgecolor="k"
)

p4 = ax2.bar(
    x - width,
    df.loc[["Seren", "Kalos"], "complete_rate_gpu_time"] * 100,
    width,
    label="Completed",
    alpha=0.8,
    linewidth=1,
    edgecolor="k",
)
p5 = ax2.bar(
    x, df.loc[["Seren", "Kalos"], "cancel_rate_gpu_time"] * 100, width, label="Canceled", alpha=0.8, linewidth=1, edgecolor="k"
)
p6 = ax2.bar(
    x + width,
    df.loc[["Seren", "Kalos"], "fail_rate_gpu_time"] * 100,
    width,
    label="Failed",
    alpha=0.8,
    linewidth=1,
    edgecolor="k",
)

autolabel(p1, ax1)
autolabel(p2, ax1)
autolabel(p3, ax1)
autolabel(p4, ax2)
autolabel(p5, ax2)
autolabel(p6, ax2)

ax1.set_xlabel(f"(a) Job Count")
ax1.set_ylabel(f"Fraction (%)")
ax1.set_xticks(x)
ax1.set_xticklabels(["Seren", "Kalos"])
ax1.set_xlim(0.5, 2.5)
ax1.set_ylim(0, 100)
ax1.grid(axis="y", linestyle=":")

ax2.set_xlabel(f"(b) GPU Time")
ax2.set_ylabel(f"Fraction (%)")
ax2.set_xticks(x)
ax2.set_xticklabels(["Seren", "Kalos"])
ax2.set_xlim(0.5, 2.5)
ax2.set_ylim(0, 100)
ax2.grid(axis="y", linestyle=":")

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, ncols=5, bbox_to_anchor=(0.18, 1.145), loc=2)

sns.despine()
fig.savefig(f"{SAVEPATH}/bar_job_state.pdf", bbox_inches="tight")

#### Treemap: Job Number Distribution

In [ ]:
print("Processing Seren")
datas = data_seren[data_seren["gpu_num"] > 0]

job_type = ["Eval", "Pretrain", "SFT", "MLLM", "Debug", "Other"]
df = pd.DataFrame(index=job_type, columns=["job_count"]).fillna(0)
df["job_count"] = df.index.map(datas.groupby("type").size()).astype(int)
df["gtime"] = df.index.map(datas.groupby("type")["gpu_time"].sum()).astype(int)

total = df["job_count"].sum()
total_gtime = df["gtime"].sum()

df["count_percent"] = df["job_count"] / total * 100
df["gtime_percent"] = df["gtime"] / total_gtime * 100

# For plotting
df["label"] = [x + f"\n{df.at[x, 'count_percent']:.1f}%" for x in list(df.index)]
df["label_gtime"] = [x + f"\n{df.at[x, 'gtime_percent']:.1f}%" for x in list(df.index)]
df["label_percent"] = [f"{df.at[x, 'count_percent']:.1f}%" for x in list(df.index)]
df["label_gtime_percent"] = [f"{df.at[x, 'gtime_percent']:.1f}%" for x in list(df.index)]
df_s = df.copy()

print("Processing Kalos")
datak = data_kalos[data_kalos["gpu_num"] > 0]

job_type = ["Eval", "Pretrain", "Debug", "Other"]
df = pd.DataFrame(index=job_type, columns=["job_count"]).fillna(0)
df["job_count"] = df.index.map(datak.groupby("type").size()).astype(int)
df["gtime"] = df.index.map(datak.groupby("type")["gpu_time"].sum()).astype(int)


total = df["job_count"].sum()
total_gtime = df["gtime"].sum()

df["count_percent"] = df["job_count"] / total * 100
df["gtime_percent"] = df["gtime"] / total_gtime * 100

# For plotting
df["label"] = [x + f"\n{df.at[x, 'count_percent']:.1f}%" for x in list(df.index)]
df["label_gtime"] = [x + f"\n{df.at[x, 'gtime_percent']:.1f}%" for x in list(df.index)]
df["label_percent"] = [f"{df.at[x, 'count_percent']:.1f}\n%" for x in list(df.index)]
df["label_gtime_percent"] = [f"{df.at[x, 'gtime_percent']:.1f}\n%" for x in list(df.index)]
df_k = df.copy()

# For plotting
df_k.at["Pretrain", "label_gtime_percent"] = df_k.at["Pretrain", "label_gtime"]
df_k.at["Eval", "label_percent"] = df_k.at["Eval", "label"]
df_k.at["Other", "label_percent"] = ""
df_k.at["Eval", "label_gtime_percent"] = " "

df_s.at["Pretrain", "label_gtime_percent"] = df_s.at["Pretrain", "label_gtime"]
df_s.at["Eval", "label_percent"] = df_s.at["Eval", "label"]
df_s.at["SFT", "label_percent"] = df_s.at["SFT", "label"]
df_s.at["Other", "label_percent"] = df_s.at["Other", "label"]
df_s.at["Pretrain", "label_percent"] = " "
df_s.at["Debug", "label_gtime_percent"] = df_s.at["Debug", "label_gtime_percent"].replace("%", "\n%")


cmp_treemap = sns.color_palette("pastel")
label = df_s.index.to_list()
df_s["color"] = cmp_treemap[: len(df_s)]

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, constrained_layout=True, figsize=(9, 4.2))
FONT = 15

###### Fig 1 ######
df_s.sort_values(by="count_percent", ascending=False, inplace=True)
squarify.plot(
    ax=ax1,
    sizes=list(df_s["job_count"].values),
    label=df_s["label_percent"],
    text_kwargs={"fontsize": FONT},
    color=df_s["color"],
    bar_kwargs={"alpha": 0.8, "linewidth": 1, "edgecolor": "k"},
)


handles, labels = ax1.get_legend_handles_labels()
handles_new = [handles[0], handles[-1], handles[1], handles[3], handles[4], handles[2]]
fig.legend(
    handles=handles_new, labels=label, ncols=6, bbox_to_anchor=(0.0, 1.135), loc=2, columnspacing=0.82, handletextpad=0.2
)

###### Fig 2 ######
df_s.sort_values(by="gtime", ascending=False, inplace=True)
squarify.plot(
    ax=ax2,
    sizes=list(df_s["gtime"].values),
    label=df_s["label_gtime_percent"],
    text_kwargs={"fontsize": FONT},
    color=df_s["color"],
    bar_kwargs={"alpha": 0.8, "linewidth": 1, "edgecolor": "k"},
)

plt.tick_params(axis="both", which="both", bottom=False, top=False, left=False, right=False)

ax1.set_xlabel(f"(a) Job Count", fontsize=16)
ax2.set_xlabel(f"(b) GPU Time", fontsize=16)


###### Fig 3 ######
df_k.sort_values(by="count_percent", ascending=False, inplace=True)
df_k["color"] = [df_s["color"][job_name] for job_name in df_k.index]

squarify.plot(
    ax=ax3,
    sizes=list(df_k["job_count"].values),
    label=df_k["label_percent"],
    text_kwargs={"fontsize": FONT},
    color=df_k["color"],
    bar_kwargs={"alpha": 0.8, "linewidth": 1, "edgecolor": "k"},
)

###### Fig 4 ######
df_k.sort_values(by="gtime", ascending=False, inplace=True)
squarify.plot(
    ax=ax4,
    sizes=list(df_k["gtime"].values),
    label=df_k["label_gtime_percent"],
    text_kwargs={"fontsize": FONT},
    color=df_k["color"],
    bar_kwargs={"alpha": 0.8, "linewidth": 1, "edgecolor": "k"},
)

ax1.annotate(
    df_s.at["Pretrain", "label"].split("\n")[1],
    xy=(97, 96),
    xytext=(90, 70),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=15,
)

ax3.annotate(
    df_k.at["Other", "label"].split("\n")[1],
    xy=(98.5, 92),
    xytext=(80, 80),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=15,
)

ax4.annotate(
    df_k.at["Eval", "label_gtime"].split("\n")[1],
    xy=(98.5, 93),
    xytext=(80, 80),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=15,
)

plt.tick_params(axis="both", which="both", bottom=False, top=False, left=False, right=False)

ax3.set_xlabel(f"(c) Job Count", fontsize=16)
ax4.set_xlabel(f"(d) GPU Time", fontsize=16, labelpad=8)

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])
ax4.set_xticks([])
ax4.set_yticks([])

ax1.text(0.015, 0.03, "Seren", transform=ax1.transAxes, size=18, fontweight="bold")
ax2.text(0.02, 0.03, "Seren", transform=ax2.transAxes, size=18, fontweight="bold")
ax3.text(0.02, 0.03, "Kalos", transform=ax3.transAxes, size=18, fontweight="bold")
ax4.text(0.02, 0.03, "Kalos", transform=ax4.transAxes, size=18, fontweight="bold")
fig.savefig(f"{SAVEPATH}/treemap_job_dist.pdf", bbox_inches="tight")

#### CDF: Duration and Queuing Delay of Different Type

In [ ]:
"""
(a) Seren Duration (b) Seren Queuing  (c) Kalos Duration (d) Kalos Queuing
"""

# Duration part
x = [2**i for i in range(0, 22)]
y_gpu_seren_other = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Other")], x_axis=x, key="duration"
)
y_gpu_seren_debug = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Debug")], x_axis=x, key="duration"
)
y_gpu_seren_pretrain = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Pretrain")], x_axis=x, key="duration"
)
y_gpu_seren_eval = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Eval")], x_axis=x, key="duration"
)
y_gpu_seren_tuning = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "SFT")], x_axis=x, key="duration"
)
y_gpu_seren_mllm = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "MLLM")], x_axis=x, key="duration"
)

y_gpu_kalos_other = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Other")], x_axis=x, key="duration"
)
y_gpu_kalos_debug = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Debug")], x_axis=x, key="duration"
)
y_gpu_kalos_pretrain = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Pretrain")], x_axis=x, key="duration"
)
y_gpu_kalos_eval = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Eval")], x_axis=x, key="duration"
)
y_gpu_kalos_tuning = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "SFT")], x_axis=x, key="duration"
)

# Queuing part
x2 = [2**i for i in range(0, 16)]
y_que_s_other = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Other")], x_axis=x2, key="queue"
)
y_que_s_debug = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Debug")], x_axis=x2, key="queue"
)
y_que_s_pretrain = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Pretrain")], x_axis=x2, key="queue"
)
y_que_s_eval = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "Eval")], x_axis=x2, key="queue"
)
y_que_s_tuning = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "SFT")], x_axis=x2, key="queue"
)
y_que_s_mllm = calculate_num_cdf_customized_xaxis(
    data_seren[(data_seren["gpu_num"] > 0) & (data_seren["type"] == "MLLM")], x_axis=x2, key="queue"
)

y_que_ali_other = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Other")], x_axis=x2, key="queue"
)
y_que_ali_debug = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Debug")], x_axis=x2, key="queue"
)
y_que_ali_pretrain = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Pretrain")], x_axis=x2, key="queue"
)
y_que_ali_eval = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "Eval")], x_axis=x2, key="queue"
)
y_que_ali_tuning = calculate_num_cdf_customized_xaxis(
    data_kalos[(data_kalos["gpu_num"] > 0) & (data_kalos["type"] == "SFT")], x_axis=x2, key="queue"
)

In [ ]:
linestyles = ["--", "-.", ":", "--", "-.", ":"]
grid_params = dict(width_ratios=[1, 1])
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, constrained_layout=True, figsize=(9, 7))

# (a) Seren Duration
ax1.plot(x, y_gpu_seren_eval, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Evaluation")
ax1.plot(x, y_gpu_seren_pretrain, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Pretrain")
ax1.plot(x, y_gpu_seren_tuning, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="SFT")
ax1.plot(x, y_gpu_seren_mllm, linestyles[0], linewidth=3, alpha=0.9, color=cmp[3], label="MLLM")
ax1.plot(x, y_gpu_seren_debug, linestyles[1], linewidth=3, alpha=0.9, color=cmp[4], label="Debug")
ax1.plot(x, y_gpu_seren_other, linestyles[2], linewidth=3, alpha=0.9, color=cmp[5], label="Other")


# (b) Seren Queuing
ax2.plot(x2, y_que_s_eval, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Evaluation")
ax2.plot(x2, y_que_s_pretrain, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Pretrain")
ax2.plot(x2, y_que_s_tuning, linestyles[2], linewidth=3, alpha=0.9, color=cmp[2], label="SFT")
ax2.plot(x2, y_que_s_mllm, linestyles[0], linewidth=3, alpha=0.9, color=cmp[3], label="MLLM")
ax2.plot(x2, y_que_s_debug, linestyles[1], linewidth=3, alpha=0.9, color=cmp[4], label="Debug")
ax2.plot(x2, y_que_s_other, linestyles[2], linewidth=3, alpha=0.9, color=cmp[5], label="Other")

# (c) Kalos Duration
ax3.plot(x, y_gpu_kalos_eval, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Evaluation")
ax3.plot(x, y_gpu_kalos_pretrain, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Pretrain")
ax3.plot(x, y_gpu_kalos_debug, linestyles[1], linewidth=3, alpha=0.9, color=cmp[4], label="Debug")
ax3.plot(x, y_gpu_kalos_other, linestyles[2], linewidth=3, alpha=0.9, color=cmp[5], label="Other")


# (d) Kalos Queuing
ax4.plot(x2, y_que_ali_eval, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Evaluation")
ax4.plot(x2, y_que_ali_pretrain, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Pretrain")
ax4.plot(x2, y_que_ali_debug, linestyles[1], linewidth=3, alpha=0.9, color=cmp[4], label="Debug")
ax4.plot(x2, y_que_ali_other, linestyles[2], linewidth=3, alpha=0.9, color=cmp[5], label="Other")

ax1.set_xlabel(f"(a) Job Duration (s)")
ax1.set_ylabel(f"CDF (%)")
ax1.set_xscale("log")
ax1.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6])
ax1.set_xlim(1, x[-1])
ax1.set_ylim(-0.5, 100.8)
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, ncols=6, bbox_to_anchor=(-0.01, 1.08), loc=2, columnspacing=0.9, handletextpad=0.2)
ax1.grid(linestyle=":")

ax2.set_xlabel(f"(b) Job Queuing Delay (s)")
ax2.set_ylabel(f"CDF (%)")
ax2.set_xscale("log")
ax2.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4])
ax2.set_xlim(1, x2[-1])
ax2.set_ylim(-0.5, 100.8)
ax2.grid(linestyle=":")

ax3.set_xlabel(f"(c) Job Duration (s)")
ax3.set_ylabel(f"CDF (%)")
ax3.set_xscale("log")
ax3.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6])
ax3.set_xlim(1, x[-1])
ax3.set_ylim(-0.5, 100.8)
ax3.grid(linestyle=":")

ax4.set_xlabel(f"(d) Job Queuing Delay (s)")
ax4.set_ylabel(f"CDF (%)")
ax4.set_xscale("log")
ax4.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4])
ax4.set_xlim(1, x2[-1])
ax4.set_ylim(-0.5, 100.8)
ax4.grid(linestyle=":")

# 1 hour and 1 day
ax1.axvline(x=3600, ls="--", alpha=0.6, c="gray", ymax=0.94, lw=1.5)
ax1.axvline(x=3600 * 24, ls="--", alpha=0.9, c="gray", ymax=0.94, lw=1.5)
ax3.axvline(x=3600, ls="--", alpha=0.6, c="gray", ymax=0.94, lw=1.5)
ax3.axvline(x=3600 * 24, ls="--", alpha=0.9, c="gray", ymax=0.94, lw=1.5)

sns.despine()
ax1.text(0.78, 0.03, "Seren", transform=ax1.transAxes, size=20, fontweight="bold")
ax2.text(0.78, 0.03, "Seren", transform=ax2.transAxes, size=20, fontweight="bold")
ax3.text(0.78, 0.03, "Kalos", transform=ax3.transAxes, size=20, fontweight="bold")
ax4.text(0.78, 0.03, "Kalos", transform=ax4.transAxes, size=20, fontweight="bold")

fig.savefig(f"{SAVEPATH}/cdf_job_duration_queue.pdf", bbox_inches="tight")

#### Box Plot: Request GPU number Different Type

In [ ]:
cmap = sns.color_palette("pastel")
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    gridspec_kw={"width_ratios": [4.2, 3]},
    constrained_layout=True,
    figsize=(9, 3.75),
)

############ Fig 1 ############
data_seren.sort_values(by="gpu_num", ascending=False, inplace=True)
data_seren["type"].replace("SFT", "SFT", inplace=True)

x_ticks = [
    "Eval",
    "Pretrain",
    "SFT",
    "MLLM",
    "Debug",
    "Other",
]

flierprops = dict(marker=".", markerfacecolor="k", markersize=2, linestyle="none")
sns.boxplot(
    x="type",
    y="gpu_num",
    data=data_seren,
    flierprops=flierprops,
    width=0.6,
    linewidth=2.2,
    saturation=2,
    palette=cmap,
    ax=ax1,
    order=x_ticks,
    boxprops=dict(alpha=1),
)
sns.color_palette("tab10")
ax1.set_xlabel("(a) Seren")
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.set_ylabel(f"Number of GPUs")
ax1.set_yscale("log")
ax1.grid(axis="y", linestyle=":")


############ Fig 2 ############
data_kalos.sort_values(by="gpu_num", ascending=False, inplace=True)
data_kalos = data_kalos[data_kalos["type"] != "SFT"]
x_ticks_k = [
    "Eval",
    "Pretrain",
    "Debug",
    "Other",
]
my_pal = [cmap[0], cmap[1], cmap[4], cmap[5]]

flierprops = dict(marker=".", markerfacecolor="k", markersize=3, linestyle="none")
sns.boxplot(
    x="type",
    y="gpu_num",
    data=data_kalos,
    flierprops=flierprops,
    width=0.6,
    linewidth=2.2,
    saturation=2,
    palette=my_pal,
    ax=ax2,
    order=x_ticks_k,
    boxprops=dict(alpha=1),
)
sns.color_palette("tab10")
ax2.set_xlabel("(b) Kalos")
ax2.set_ylabel(None)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.set_yscale("log")
ax2.grid(axis="y", linestyle=":")

sns.despine()
fig.savefig(f"{SAVEPATH}/box_gpu_num.pdf", bbox_inches="tight")

#### CDF: Resource Utilization

In [ ]:
with open(f"{PKLPATH}/util_gpu_seren.pkl", "rb") as file:
    _, _, x2, y2, x3, y3, x4, y4, x5, y5 = pickle.load(file)
with open(f"{PKLPATH}/util_gpu_kalos_full.pkl", "rb") as file:
    _, _, x2_k, y2_k, x3_k, y3_k, x4_k, y4_k, x5_k, y5_k = pickle.load(file)
with open(f"{PKLPATH}/util_cpu_mem_seren.pkl", "rb") as file:
    x6, y6, x7, y7 = pickle.load(file)
with open(f"{PKLPATH}/util_cpu_mem_kalos.pkl", "rb") as file:
    x6_k, y6_k, x7_k, y7_k = pickle.load(file)
with open(f"{PKLPATH}/ib_seren.pkl", "rb") as file:
    x8, y8, x9, y9 = pickle.load(file)

x8 = x8 / x8.max() * 100
x9 = x9 / x9.max() * 100

linestyles = ["--", ":", "--", "-.", ":"]
grid_params = dict(width_ratios=[1, 1])
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, constrained_layout=True, figsize=(9, 7))

############ Fig 1: SM, Occupancy ############
ax1.plot(x3, y3, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren SM Activity")
ax1.plot(x5, y5, linestyles[1], linewidth=3, alpha=0.9, color=cmp[0], label="Seren Occupancy")
ax1.plot(x3_k, y3_k, linestyles[0], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos SM Activity")
ax1.plot(x5_k, y5_k, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos Occupancy")

############ Fig 2: CPU mem usage, GPU mem usage ############
ax2.plot(x7, y7, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren CPU Mem")
ax2.plot(x2, y2, linestyles[1], linewidth=3, alpha=0.9, color=cmp[0], label="Seren GPU Mem")
ax2.plot(x7_k, y7_k, linestyles[0], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos CPU Mem")
ax2.plot(x2_k, y2_k, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos GPU Mem")

############ Fig 3: CPU util ############
ax3.plot(x6, y6, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax3.plot(x6_k, y6_k, linestyles[0], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")

############ Fig 4: IB send, receive ############
ax4.plot(x8, y8, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="IB Send")
ax4.plot(x9, y9, linestyles[1], linewidth=3, alpha=0.9, color=cmp[0], label="IB Receive")

ax1.set_xlabel(f"(a) GPU DCGM Metric (%)")
ax1.set_ylabel(f"CDF (%)")
ax1.set_xlim(-0.8, 100.8)
ax1.set_ylim(0, 100.8)
ax1.set_xticks([0, 25, 50, 75, 100])
ax1.grid(linestyle=":")

ax2.set_xlabel(f"(b) Memory Footprint (%)")
ax2.set_ylabel(f"CDF (%)")
ax2.set_xlim(-0.8, 100.8)
ax2.set_xticks([0, 25, 50, 75, 100])
ax2.set_ylim(0, 100.8)
ax2.grid(linestyle=":")

ax3.set_xlabel(f"(c) CPU Utilization (%)")
ax3.set_ylabel(f"CDF (%)")
ax3.set_xlim(-0.8, 100.8)
ax3.set_xticks([0, 25, 50, 75, 100])
ax3.set_ylim(0, 100.8)
ax3.legend(loc="lower right")
ax3.grid(linestyle=":")

ax4.set_xlabel(f"(d) Network (%)")
ax4.set_ylabel(f"CDF (%)")
ax4.set_xlim(-0.8, 100.8)
ax4.set_xticks([0, 25, 50, 75, 100])
ax4.set_ylim(0, 100.8)
ax4.legend(loc="lower right")
ax4.grid(linestyle=":")
sns.despine()


S = mpatches.Patch(facecolor=cmp[0], alpha=0.9)
K = mpatches.Patch(facecolor=cmp[1], alpha=0.9)
A = (Line2D([0], [0], color="black", lw=3, ls="--"),)
B = (Line2D([0], [0], color="black", lw=3, ls=":"),)

legend1 = ax1.legend([S, K], ["Seren", "Kalos"], bbox_to_anchor=(0.5, 0.62), loc=2, ncol=1, fontsize=17, frameon=False)

ax1.add_artist(legend1)

ax1.legend([A, B], ["SM Activity", "TC Activity"], bbox_to_anchor=(0.3, 0.36), loc=2, ncol=1)

ax2.legend([A, B], ["CPU Memory", "GPU Memory"], bbox_to_anchor=(0.25, 0.36), loc=2, ncol=1)

fig.savefig(f"{SAVEPATH}/cdf_resource_util.pdf", bbox_inches="tight")

#### CDF: Temperature

In [ ]:
# We use August data for GPU temperature and power
with open(f"{PKLPATH}/gpu_temp_seren.pkl", "rb") as file:
    x, y1, x2, y2 = pickle.load(file)
with open(f"{PKLPATH}/gpu_temp_kalos.pkl", "rb") as file:
    x1_k, y1_k, x2_k, y2_k = pickle.load(file)
with open(f"{PKLPATH}/gpu_power_seren.pkl", "rb") as file:
    x3, y3 = pickle.load(file)
with open(f"{PKLPATH}/gpu_power_kalos.pkl", "rb") as file:
    x3_k, y3_k = pickle.load(file)

linestyles = ["-", ":", ":", "-"]
fig, ax1 = plt.subplots(ncols=1, nrows=1, constrained_layout=True, figsize=(5, 3.75))

############ Fig 1: Temperature ############
ax1.plot(x, y1, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren GPU Temp")
ax1.plot(x2, y2, linestyles[1], linewidth=3, alpha=0.9, color=cmp[0], label="Seren GPU Mem Temp")
ax1.plot(x1_k, y1_k, linestyles[0], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos GPU Temp")
ax1.plot(x2_k, y2_k, linestyles[1], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos GPU Mem Temp")

ax1.set_xlabel(f"Temperature (°C)")
ax1.set_ylabel(f"CDF (%)")
ax1.set_xlim(20, 85)
ax1.set_ylim(0, 100.8)
ax1.grid(linestyle=":")

S = mpatches.Patch(facecolor=cmp[0], alpha=0.9)
K = mpatches.Patch(facecolor=cmp[1], alpha=0.9)
A = (Line2D([0], [0], color="black", lw=3, ls="-"),)
B = (Line2D([0], [0], color="black", lw=3, ls=":"),)

legend1 = ax1.legend([S, K], ["Seren", "Kalos"], bbox_to_anchor=(0.6, 0.62), loc=2, ncol=1, fontsize=17, frameon=False)

ax1.add_artist(legend1)

ax1.legend(
    [A, B],
    ["GPU Temp.", "GMem Temp."],
    bbox_to_anchor=(0.4, 0.32),
    loc=2,
    ncol=1,
    fontsize=17,
)

sns.despine()
fig.savefig(f"{SAVEPATH}/cdf_temperature.pdf", bbox_inches="tight")

#### CDF: Power

In [ ]:
with open(f"{PKLPATH}/server_power.pkl", "rb") as file:
    x1, y1, x2, y2 = pickle.load(file)
with open(f"{PKLPATH}/gpu_power_seren.pkl", "rb") as file:
    x3, y3 = pickle.load(file)
with open(f"{PKLPATH}/gpu_power_kalos.pkl", "rb") as file:
    x3_k, y3_k = pickle.load(file)

In [ ]:
linestyles = ["--", ":", ":", "-"]
grid_params = dict(width_ratios=[1, 1])
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, constrained_layout=True, figsize=(9, 3.75))

############ Fig 1: GPU power ############
ax1.plot(x3, y3, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="Seren")
ax1.plot(x3_k, y3_k, linestyles[0], linewidth=3, alpha=0.9, color=cmp[1], label="Kalos")
ax1.axvline(x=400, ls="--", alpha=0.6, c="gray", ymax=100, lw=1.5)
ax1.annotate(
    "A100 TDP",
    xy=(400, 33),
    xytext=(420, 20),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=16,
)
ax1.annotate(
    "Max=600",
    xy=(600, 100),
    xytext=(430, 85),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=16,
)

############ Fig 2: Server power ############
ax2.plot(x1, y1, linestyles[0], linewidth=3, alpha=0.9, color=cmp[0], label="GPU Node")
ax2.plot(x2, y2, linestyles[1], linewidth=3, alpha=0.9, color=cmp[0], label="CPU Node")
ax2.annotate(
    "Max=960",
    xy=(960, 100),
    xytext=(1200, 90),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=16,
)
ax2.annotate(
    "Max=6550",
    xy=(6550, 100),
    xytext=(4500, 70),
    arrowprops=dict(facecolor="black", width=2.5, headwidth=8),
    color="black",
    fontsize=16,
)

ax1.set_xlabel(f"(a) GPU Power (W)")
ax1.set_ylabel(f"CDF (%)")
ax1.set_xlim(-0.8, 610)
ax1.set_ylim(0, 100.8)
ax1.legend()
ax1.grid(linestyle=":")
ax1.xaxis.set_minor_locator(matplotlib.ticker.FixedLocator([60]))
ax1.xaxis.set_minor_formatter(matplotlib.ticker.FixedFormatter([60]))
ax1.tick_params(axis="x", which="minor", labelsize=15)

ax2.set_xlabel(f"(b) Server Power in Seren (W)")
ax2.set_ylabel(f"CDF (%)")
ax2.set_xlim(-0.8, x1.max())
ax2.set_ylim(0, 100.8)
ax2.legend(loc="lower right")
ax2.grid(linestyle=":")
ax2.xaxis.set_minor_locator(matplotlib.ticker.FixedLocator([520]))
ax2.xaxis.set_minor_formatter(matplotlib.ticker.FixedFormatter([520]))
ax2.tick_params(axis="x", which="minor", labelsize=15)
sns.despine()

fig.savefig(f"{SAVEPATH}/cdf_power.pdf", bbox_inches="tight")